In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd

In [3]:
ROOT = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen"

In [4]:
df_indobert = pd.read_csv(f'{ROOT}/nested/results-prediction/indobert.csv')
df_bilstm = pd.read_csv(f'{ROOT}/nested/results-prediction/bilstm.csv')
df_crf = pd.read_csv(f'{ROOT}/nested/results-prediction/crf.csv')

In [5]:
def get_level_token_df(tag, max_lev=2, level=0) : 

  new_tag = []
  tag_per_level = tag.split('+')
  for lev in range(max_lev) :
    if lev + 1 > len(tag_per_level) : 
      new_tag.append(tag_per_level[-1])
    else : 
      new_tag.append(tag_per_level[lev])
  return new_tag[level]


In [6]:
df_indobert

,sentence #,juz,no_ayat,sent,tag,pred
0,2,1,2,Segala,O+O,O+O
1,2,1,2,puji,O+O,O+O
2,2,1,2,bagi,O+O,O+O
3,2,1,2,Allah,O+O,O+O
4,2,1,2,Tuhan,O+O,O+O
...,...,...,...,...,...,...
7018,944,6,155,bertakwalah,O+O,O+O
7019,944,6,155,agar,O+O,O+O
7020,944,6,155,kamu,O+O,O+O
7021,944,6,155,diberi,O+O,O+O


In [7]:
df_crf.pred.apply(lambda x : get_level_token_df(x, level=0))

0       O
1       O
2       O
3       O
4       O
       ..
7018    O
7019    O
7020    O
7021    O
7022    O
Name: pred, Length: 7023, dtype: object

In [8]:
df_level_1 = df_indobert[['sentence #', 'juz', 'no_ayat', 'sent']].copy()

df_level_1['true'] = df_indobert.tag.apply(lambda x : get_level_token_df(x, level=0))
df_level_1['pred_indobert'] = df_indobert.pred.apply(lambda x : get_level_token_df(x, level=0))
df_level_1['pred_bilstm'] = df_bilstm.pred.apply(lambda x : get_level_token_df(x, level=0))
df_level_1['pred_crf'] = df_crf.pred.apply(lambda x : get_level_token_df(x, level=0))


In [9]:
df_level_1[(df_level_1.juz == 2) & (df_level_1.no_ayat == 99)]

,sentence #,juz,no_ayat,sent,true,pred_indobert,pred_bilstm,pred_crf
615,106,2,99,Dan,O,O,O,O
616,106,2,99,sesungguhnya,O,O,O,O
617,106,2,99,Kami,O,O,O,O
618,106,2,99,telah,O,O,O,O
619,106,2,99,menurunkan,O,O,O,O
620,106,2,99,kepadamu,O,O,O,O
621,106,2,99,ayat-ayat,O,O,O,O
622,106,2,99,yang,O,O,O,O
623,106,2,99,jelas,O,O,O,O
624,106,2,99,dan,O,O,O,O


In [10]:
df_level_1.to_csv(f'{ROOT}/nested/results-prediction/level_1.csv', index=False)

In [11]:
df_level_2 = df_indobert[['sentence #', 'juz', 'no_ayat', 'sent']].copy()

df_level_2['true'] = df_indobert.tag.apply(lambda x : get_level_token_df(x, level=1))
df_level_2['pred_indobert'] = df_indobert.pred.apply(lambda x : get_level_token_df(x, level=1))
df_level_2['pred_bilstm'] = df_bilstm.pred.apply(lambda x : get_level_token_df(x, level=1))
df_level_2['pred_crf'] = df_crf.pred.apply(lambda x : get_level_token_df(x, level=1))


In [12]:
df_level_2.to_csv(f'{ROOT}/nested/results-prediction/level_2.csv', index=False)

In [13]:
count_1 = 0
count_2 = 0
for sentence in df_indobert['sentence #'].unique() : 
  tags_1 = df_level_1[df_level_1['sentence #'] == sentence].true.to_list()
  tags_2 = df_level_2[df_level_2['sentence #'] == sentence].true.to_list()


  lev_2_flag = False
  for i in range(len(tags_1)) :
    if tags_1[i] == "B-PER" :
      lev_2_flag = False
      count_1 += 1
      count_2 += 1

    if not lev_2_flag and tags_1[i] == "O" and tags_2[i] == "I-PER" : 
      lev_2_flag = True
      count_2 += 1



In [14]:
tags_2

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [15]:
count_1

222

In [16]:
count_2

252

In [17]:
nested_ent = count_2 - count_1
nested_ent

30